# Getting the Data From Wikipedia

Importing the Pandas and grabbing the table from the wiki page

In [1]:
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(url, header=0,keep_default_na=False)
tdf = table[0]
tdf.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Fixing up the column names

In [3]:
tdf.columns = ['PostalCode','Borough','Neighborhood'] 
tdf.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Getting rid of unassigned boroughs

In [4]:
tdf1 = tdf.query('Borough != "Not assigned"').reset_index(drop=True) 
tdf1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


If there is more than one neighborhood in the same postalcode, combine the neighborhood separated by commas

In [5]:
tdf2=tdf1.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
tdf2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [6]:
tdf2.loc[tdf2['Neighborhood'] == 'Not assigned', 'Neighborhood' ] = tdf2['Borough']
tdf2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


To see number of rows in the data frame

In [7]:
tdf2.shape[0]

103

Getting the geographical coordinates of each postal code

In [8]:
latlong = pd.read_csv("http://cocl.us/Geospatial_data/") 
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Rename "Postal Code" with "PostalCode"

In [9]:
latlong.columns = ['PostalCode','Latitude','Longitude']
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We merge the two tables with latlong on the right

In [10]:
tdf2 = pd.merge(left=tdf1, right=latlong, on="PostalCode", how="right")
tdf2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


# Now Exploring Toronto

In [15]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

# create map of York using latitude and longitude values

map_Toronto = folium.Map(location=[43.761539, -79.411079], zoom_start=11)

# add markers to map
for lat, lng, bor, nei in zip(tdf2['Latitude'], tdf2['Longitude'], tdf2['Borough'], tdf2['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Toronto)  
    
map_Toronto

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         704 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.0-py_0 conda-forge
    branca:  0.3.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 1

Examine Neighborhoods in Scarborough

In [16]:
Scarborough_data = tdf2[tdf2['Borough'] == 'Scarborough'].reset_index(drop=True)
map_scarborough = folium.Map(location=[43.773077, -79.257774], zoom_start=11)
for lat, lng, bor, nei in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'],Scarborough_data['Borough'], Scarborough_data['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarborough)  
    
map_scarborough

Define Foursquare Credentials and Version

In [17]:
###Removing CLIENT_ID and CLIENT_SECRET after run

CLIENT_ID = 'KIBCQRARRRRSUKVEUP2JVQBSKUYXZPYAOITPLBGFR44GHXAL' # your Foursquare ID
CLIENT_SECRET = 'IZMFY5GDVGEPNGHEPZADKISQQCNOVBTQHDX3LP0PMWFQOD2A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Examine Neighborhoods in Scarborough, lat-long 43.773077, -79.257774

In [18]:
Scarborough_data = tdf2[tdf2['Borough'] == 'Scarborough'].reset_index(drop=True)
map_scarborough = folium.Map(location=[43.773077, -79.257774], zoom_start=11)
for lat, lng, bor, nei in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'],Scarborough_data['Borough'], Scarborough_data['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarborough)  
    
map_scarborough

We find nearby venues in Scarborough

In [19]:
import requests
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)


scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Rouge
Malvern
Highland Creek
Rouge Hill
Port Union
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park
Ionview
Kennedy Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Birch Cliff
Cliffside West
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Clarks Corners
Sullivan
Tam O'Shanter
Agincourt North
L'Amoreaux East
Milliken
Steeles East
L'Amoreaux West
Upper Rouge


Cluster and visulaize neighborhoods

In [20]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
Agincourt North,2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
Clairlea,10,10,10,10,10,10
Clarks Corners,10,10,10,10,10,10
Cliffcrest,2,2,2,2,2,2
Cliffside,2,2,2,2,2,2
Cliffside West,4,4,4,4,4,4


In [21]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,...,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000,0.000,0.0,0.200000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.20,0.00,0.0,0.000,0.0
1,Agincourt North,0.0,0.000,0.000,0.000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.5,0.000000,0.000000,0.00,0.00,0.0,0.000,0.0
2,Birch Cliff,0.0,0.000,0.000,0.000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.00,0.0,0.000,0.0
3,Cedarbrae,0.0,0.125,0.125,0.125,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.0,0.125,0.0
4,Clairlea,0.0,0.000,0.200,0.000,0.0,0.000000,0.000000,0.2,0.100000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.1,0.000,0.0
5,Clarks Corners,0.0,0.000,0.000,0.100,0.0,0.000000,0.000000,0.0,0.000000,...,0.100000,0.200000,0.0,0.000000,0.000000,0.00,0.00,0.0,0.100,0.0
6,Cliffcrest,0.5,0.000,0.000,0.000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000,0.0
7,Cliffside,0.5,0.000,0.000,0.000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000,0.0
8,Cliffside West,0.0,0.000,0.000,0.000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.00,0.0,0.000,0.0
9,Dorset Park,0.0,0.000,0.000,0.000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000,0.2


In [22]:
import numpy as np
num_top_venues = 17

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Chinese Restaurant,Grocery Store
1,Agincourt North,Playground,Park,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Caribbean Restaurant,Chinese Restaurant,Gym
2,Birch Cliff,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Gym,Caribbean Restaurant
3,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Convenience Store,Grocery Store,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Clothing Store,College Stadium
4,Clairlea,Bakery,Bus Line,Metro Station,Soccer Field,Fast Food Restaurant,Intersection,Bus Station,Park,Vietnamese Restaurant,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Chinese Restaurant
5,Clarks Corners,Pizza Place,Thai Restaurant,Fried Chicken Joint,Bank,Fast Food Restaurant,Italian Restaurant,Pharmacy,Noodle House,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Caribbean Restaurant
6,Cliffcrest,American Restaurant,Motel,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
7,Cliffside,American Restaurant,Motel,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
8,Cliffside West,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Gym,Caribbean Restaurant
9,Dorset Park,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Bank,College Stadium,General Entertainment,Athletics & Sports,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Bar


Cluster the neighborhood Scarborough

In [23]:
from sklearn.cluster import KMeans


# Number of clusters 4
# Using k-means to cluster the neighborhood into 4 clusters.
kclusters = 4

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 0, 1, 2, 1, 0, 1,
       1, 3, 2, 3, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [25]:
scarborough_merged = Scarborough_data[0:36]

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


scarborough_merged

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,M1B,Scarborough,Rouge,43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,Grocery Store,...,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
1,M1B,Scarborough,Malvern,43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,Grocery Store,...,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497,1,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,...,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Thai Restaurant,Chinese Restaurant,Caribbean Restaurant
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497,1,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,...,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Thai Restaurant,Chinese Restaurant,Caribbean Restaurant
4,M1C,Scarborough,Port Union,43.784535,-79.160497,1,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,...,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Thai Restaurant,Chinese Restaurant,Caribbean Restaurant
5,M1E,Scarborough,Guildwood,43.763573,-79.188711,1,Mexican Restaurant,Electronics Store,Intersection,Rental Car Location,...,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Clothing Store
6,M1E,Scarborough,Morningside,43.763573,-79.188711,1,Mexican Restaurant,Electronics Store,Intersection,Rental Car Location,...,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Clothing Store
7,M1E,Scarborough,West Hill,43.763573,-79.188711,1,Mexican Restaurant,Electronics Store,Intersection,Rental Car Location,...,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Clothing Store
8,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Gym,...,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Athletics & Sports,Bakery,Bank,...,Thai Restaurant,Convenience Store,Grocery Store,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Clothing Store,College Stadium


In [26]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.773077, -79.257774], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


Examine the clusters

Clutser 0

In [27]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
1,Malvern,0,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
16,Oakridge,0,Bakery,Bus Line,Metro Station,Soccer Field,Fast Food Restaurant,Intersection,Bus Station,Park,Vietnamese Restaurant,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Chinese Restaurant
20,Birch Cliff,0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Gym,Caribbean Restaurant
27,Agincourt,0,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Chinese Restaurant,Grocery Store
29,Sullivan,0,Pizza Place,Thai Restaurant,Fried Chicken Joint,Bank,Fast Food Restaurant,Italian Restaurant,Pharmacy,Noodle House,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Caribbean Restaurant


Clutser 1

In [28]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,Rouge,1,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
2,Highland Creek,1,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Thai Restaurant,Chinese Restaurant,Caribbean Restaurant
3,Rouge Hill,1,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Thai Restaurant,Chinese Restaurant,Caribbean Restaurant
4,Port Union,1,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Thai Restaurant,Chinese Restaurant,Caribbean Restaurant
5,Guildwood,1,Mexican Restaurant,Electronics Store,Intersection,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Clothing Store
6,Morningside,1,Mexican Restaurant,Electronics Store,Intersection,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Clothing Store
7,West Hill,1,Mexican Restaurant,Electronics Store,Intersection,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Clothing Store
8,Woburn,1,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
9,Cedarbrae,1,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Convenience Store,Grocery Store,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Clothing Store,College Stadium
10,Scarborough Village,1,Playground,Convenience Store,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium,Coffee Shop,Chinese Restaurant,Gym,Caribbean Restaurant,Café


Cluster 2

In [29]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
18,Cliffside,2,American Restaurant,Motel,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
24,Wexford Heights,2,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Bank,College Stadium,General Entertainment,Athletics & Sports,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Bar


Cluster 3

In [30]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
13,Kennedy Park,3,Discount Store,Hobby Shop,Coffee Shop,Bus Station,Department Store,Convenience Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,College Stadium,Vietnamese Restaurant,Gym,Chinese Restaurant,Caribbean Restaurant
23,Scarborough Town Centre,3,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Bank,College Stadium,General Entertainment,Athletics & Sports,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Bar
25,Maryvale,3,Middle Eastern Restaurant,Smoke Shop,Bakery,Breakfast Spot,Vietnamese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Clothing Store


# The End